# Notebook para Detección de Fugas usando Escalogramas CWT, ELIS y ResNet-18

## 1. Instalación de Dependencias e Importación de Librerías

In [ ]:
# Instalación de dependencias para wavelet denoising
!pip install PyWavelets

# Instalación de dependencias para fCWT
!git clone https://github.com/fastlib/fCWT.git
!pip install fCWT
!apt-get update
!apt-get install libfftw3-single3 -y

# Instalación de dependencias para procesamiento de imágenes
!pip install opencv-python

# Importación de bibliotecas necesarias
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import matplotlib.cm as cm
import scipy.signal as sig
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
import h5py
from tqdm.notebook import tqdm
import random
import pywt
import fcwt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import sys
import time
import datetime
import cv2

# Para visualización de imágenes
from IPython.display import display

# Importar la clase WaveletDenoising (debe estar en un archivo denoising.py en el path)
from denoising import WaveletDenoising

# Montar Google Drive (para Colab)
from google.colab import drive
drive.mount('/content/drive')

# Configuraciones globales para visualización
plt.style.use('seaborn-v0_8-whitegrid')

## 2. Cargar y Visualizar Datos

In [ ]:
# Cambiando la ruta para acceder a los datos en Google Drive
data_dir = '/content/drive/MyDrive/Tesis/Accelerometer_Dataset/Branched'
original_sr = 25600  # in Hz
signal_sr = 25600  # in Hz
downsample_factor = original_sr//signal_sr

# Modo de clasificación: 'five_classes' o 'binary'
classification_mode = 'five_classes'  # Cambiar según necesidad

# Definir diccionarios de etiquetas según el modo de clasificación
if classification_mode == 'five_classes':
    label_codes_dict = {'Circumferential Crack': 0, 'Gasket Leak': 1, 'Longitudinal Crack': 2, 'No-leak': 3, 'Orifice Leak': 4}
else:  # binary
    label_codes_dict = {'Leak': 0, 'No-leak': 1}

# Esta función elimina el archivo .DS_Store si existe en la ruta definida
def remove_DS_store_file(path):
    # Buscar tanto .DS_Store como .DS_store (diferencias de capitalización)
    for ds_name in ['.DS_Store', '.DS_store']:
        ds_store_file_location = os.path.join(path, ds_name)
        if os.path.isfile(ds_store_file_location):
            os.remove(ds_store_file_location)

def load_accelerometer_data(data_dir, sample_rate, downsample_factor, label_codes, mode='five_classes', fraction_to_include=1):
    """
    Carga datos de acelerómetro con opción para clasificación binaria o multiclase

    Args:
        data_dir: Directorio donde se encuentran los datos
        sample_rate: Tasa de muestreo deseada
        downsample_factor: Factor de submuestreo
        label_codes: Diccionario de códigos de etiquetas
        mode: 'five_classes' o 'binary'
        fraction_to_include: Fracción de frames a incluir

    Returns:
        signals: Lista de señales
        labels: Lista de etiquetas
    """
    # Eliminar .DS_Store si existe en data_dir
    remove_DS_store_file(data_dir)

    signals = []
    labels = []

    # Para el modo binario, necesitamos contabilizar cuántos frames hay por cada tipo de fuga
    leak_counts = {
        'Circumferential Crack': 0,
        'Gasket Leak': 0,
        'Longitudinal Crack': 0,
        'Orifice Leak': 0
    }
    leak_signals = {
        'Circumferential Crack': [],
        'Gasket Leak': [],
        'Longitudinal Crack': [],
        'Orifice Leak': []
    }
    no_leak_signals = []
    no_leak_labels = []

    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)

        # Verificar que sea un directorio antes de procesarlo
        if not os.path.isdir(label_dir):
            print(f"Omitiendo {label_dir} porque no es un directorio")
            continue

        # Eliminar .DS_Store si existe
        remove_DS_store_file(label_dir)

        for file in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file)

            # Verificar que sea un archivo
            if not os.path.isfile(file_path):
                continue

            # Cargar el archivo csv
            accelerometer_signal_df = pd.read_csv(file_path, index_col=False)

            # Submuestrear seleccionando cada n-ésima fila
            accelerometer_signal_df = accelerometer_signal_df.iloc[::downsample_factor, :]
            accelerometer_signal_df = accelerometer_signal_df.reset_index(drop=True)

            # Obtener 30 segundos de datos
            accelerometer_signal = accelerometer_signal_df['Value'][0:(sample_rate*30)]

            # Generar un vector con el índice de inicio para cada frame de 1 segundo
            sample_indexes = np.linspace(0,len(accelerometer_signal)-sample_rate,len(accelerometer_signal)//sample_rate)

            # Obtener el número de frames de señal
            signal_frames_number = fraction_to_include*len(sample_indexes)
            signal_frames_counter = 0

            # Generar frames de señal de 1 segundo a partir de la señal original
            for signal_frame in sample_indexes:
                accelerometer_signal_frame = accelerometer_signal[int(signal_frame):int(signal_frame+sample_rate)]
                signal_frames_counter+=1

                if signal_frames_counter > signal_frames_number:
                    break

                if len(accelerometer_signal_frame) != sample_rate:
                    continue

                if mode == 'five_classes':
                    # Guardar directamente para clasificación de 5 clases
                    signals.append(accelerometer_signal_frame)
                    labels.append(label_codes[label])
                else:  # modo binario
                    # Para modo binario, guardamos las señales según su tipo
                    if label == 'No-leak':
                        no_leak_signals.append(accelerometer_signal_frame)
                        no_leak_labels.append(1)  # 1 para No-leak en modo binario
                    else:
                        # Guardar en la categoría correspondiente
                        leak_signals[label].append(accelerometer_signal_frame)
                        leak_counts[label] += 1

    # Si estamos en modo binario, equilibramos el dataset
    if mode == 'binary':
        # Encontrar la cantidad mínima de ejemplos por tipo de fuga
        min_count_per_leak_type = min(leak_counts.values()) if leak_counts else 0

        # Calcular cuántos ejemplos necesitamos de cada tipo para equilibrar con No-leak
        if len(no_leak_signals) > 0 and len(leak_counts) > 0:
            total_leak_samples_needed = len(no_leak_signals)
            samples_per_leak_type = total_leak_samples_needed // len(leak_counts)

            # Asegurar que no tomamos más muestras de las disponibles
            samples_per_leak_type = min(samples_per_leak_type, min_count_per_leak_type)

            # Seleccionar muestras equilibradas de cada tipo de fuga
            balanced_leak_signals = []
            for leak_type in leak_signals:
                if leak_signals[leak_type]:
                    # Tomar una muestra aleatoria del tamaño necesario
                    selected_signals = random.sample(leak_signals[leak_type],
                                                    min(samples_per_leak_type, len(leak_signals[leak_type])))
                    balanced_leak_signals.extend(selected_signals)

            # Crear etiquetas para las señales de fuga (0 para Leak en modo binario)
            balanced_leak_labels = [0] * len(balanced_leak_signals)

            # Combinar todo
            signals = balanced_leak_signals + no_leak_signals
            labels = balanced_leak_labels + no_leak_labels

            print(f"Modo binario: {len(balanced_leak_labels)} muestras de fuga, {len(no_leak_labels)} muestras sin fuga")

    return signals, labels

# Cargar los datos desde Google Drive
signals_lst, labels_lst = load_accelerometer_data(
    data_dir,
    signal_sr,
    downsample_factor,
    label_codes_dict,
    mode=classification_mode,
    fraction_to_include=1
)

signals_dict = {'training': [], 'testing': []}
labels_dict = {'training': [], 'testing': []}

# Generar diccionarios con subconjuntos de entrenamiento y prueba a partir de los datos cargados
signals_dict['training'], signals_dict['testing'], labels_dict['training'], labels_dict['testing'] = train_test_split(
    signals_lst,
    labels_lst,
    test_size=0.2,
    random_state=53
)

# Imprimir información sobre el dataset resultante:
print(f'Data Directory: {data_dir}')
print(f'Sample Rate: {signal_sr} Hz')
print(f'Classification Mode: {classification_mode}')
print(f'Number of signals (training, testing): ({len(signals_dict["training"])}, {len(signals_dict["testing"])})')
print(f'Number of labels (training, testing): ({len(labels_dict["training"])}, {len(labels_dict["testing"])})')
print(f'Number of samples per signal: {len(signals_dict["training"][np.random.randint(0,len(signals_dict["training"]))])}')

# Graficar algunas de las señales resultantes
plt.figure(figsize=(20, 20))
rows = 5
cols = 2
n = rows * cols
random_index = []

for i in range(n):
    plt.subplot(rows, cols, i+1)
    random_index.append(np.random.randint(0, len(signals_dict['training'])))
    plt.plot(signals_dict['training'][random_index[i]])

    # Obtener el nombre de la etiqueta según el modo de clasificación
    if classification_mode == 'five_classes':
        label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(labels_dict['training'][random_index[i]])]
    else:
        label_name = 'Leak' if labels_dict['training'][random_index[i]] == 0 else 'No-leak'

    plt.title(label_name)
    plt.grid()

## 3. Normalización y Denoising con Wavelet

In [ ]:
def wavelet_denoise(signals_dict, labels_dict):
    """
    Normaliza y aplica denoising wavelet a las señales

    Args:
        signals_dict: Diccionario con señales de entrenamiento y prueba
        labels_dict: Diccionario con etiquetas de entrenamiento y prueba

    Returns:
        wavelet_denoised_signals: Diccionario con señales procesadas
        labels_dict: Diccionario con etiquetas
    """
    # Crear un objeto de la clase WaveletDenoising
    wd = WaveletDenoising(normalize=True,
                      wavelet='sym3',
                      level=4,
                      thr_mode='soft',
                      method="universal")

    # Crear un nuevo diccionario para almacenar los coeficientes calculados:
    wavelet_denoised_signals = {'training': [], 'testing': []}

    for key, signals_subset in signals_dict.items():
        for signal_element in tqdm(signals_subset, desc=f"Denoising {key} signals"):
            # Denoising de la señal usando el método wavelet denoising
            denoised_signal = wd.fit(signal_element)

            # Almacenar las señales denoised en el nuevo diccionario
            wavelet_denoised_signals[key].append(denoised_signal)

    return wavelet_denoised_signals, labels_dict

# Denoising de las señales en los diccionarios de entrenamiento y prueba
wavelet_denoised_signals_dict, labels_dict = wavelet_denoise(signals_dict=signals_dict, labels_dict=labels_dict)

# Imprimir información sobre el dataset resultante:
print(f'Number of signals (training, testing): ({len(wavelet_denoised_signals_dict["training"])}, {len(wavelet_denoised_signals_dict["testing"])})')
print(f'Number of labels (training, testing): ({len(labels_dict["training"])}, {len(labels_dict["testing"])})')
print(f'Number of samples per signal: {len(wavelet_denoised_signals_dict["training"][0])}')

# Graficar algunas de las señales resultantes
plt.figure(figsize=(20, 20))
rows = 5
cols = 2
n = rows * cols

for i in range(n):
    plt.subplot(rows, cols, i+1)
    plt.plot(wavelet_denoised_signals_dict['training'][random_index[i]])

    # Obtener el nombre de la etiqueta según el modo de clasificación
    if classification_mode == 'five_classes':
        label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(labels_dict['training'][random_index[i]])]
    else:
        label_name = 'Leak' if labels_dict['training'][random_index[i]] == 0 else 'No-leak'

    plt.title(label_name)
    plt.grid()

## 3.2 Normalización de señales post-wavelet denoising

In [ ]:
# Normalización de señales post-wavelet denoising

def normalize_signals(signals_dict):
    """
    Normaliza cada señal del diccionario a un rango de 0 a 1

    Args:
        signals_dict: Diccionario con señales a normalizar

    Returns:
        normalized_signals: Diccionario con señales normalizadas
    """
    # Crear un nuevo diccionario para almacenar las señales normalizadas:
    normalized_signals = {'training': [], 'testing': []}

    for key, signals_subset in signals_dict.items():
        for signal in tqdm(signals_subset, desc=f"Normalizando {key} signals"):
            # Encontrar el valor mínimo y máximo para cada señal
            min_val = np.min(signal)
            max_val = np.max(signal)

            # Evitar división por cero
            if max_val > min_val:
                # Normalizar la señal entre 0 y 1
                normalized_signal = (signal - min_val) / (max_val - min_val)
            else:
                # Si todos los valores son iguales, asignar 0.5 a todos
                normalized_signal = np.ones_like(signal) * 0.5

            # Almacenar la señal normalizada
            normalized_signals[key].append(normalized_signal)

    return normalized_signals

# Ejecutar la normalización en las señales con denoising
print("Normalizando señales procesadas con wavelet denoising...")
normalized_signals_dict = normalize_signals(wavelet_denoised_signals_dict)

# Imprimir información sobre el dataset normalizado:
print(f'Number of signals (training, testing): ({len(normalized_signals_dict["training"])}, {len(normalized_signals_dict["testing"])})')
print(f'Number of samples per signal: {len(normalized_signals_dict["training"][0])}')

# Comprobar rango de valores
for key in normalized_signals_dict:
    sample_signal = normalized_signals_dict[key][0]
    print(f"Rango de valores en {key}: [{np.min(sample_signal):.4f}, {np.max(sample_signal):.4f}]")

# Visualizar comparación de señales originales y normalizadas
plt.figure(figsize=(20, 15))
rows = 3
cols = 2
sample_indices = random_index[:3]  # Usar los mismos índices que antes

for i, idx in enumerate(sample_indices):
    # Señal con denoising (sin normalizar)
    plt.subplot(rows, cols, i*2+1)
    plt.plot(wavelet_denoised_signals_dict['training'][idx])

    if classification_mode == 'five_classes':
        label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(labels_dict['training'][idx])]
    else:
        label_name = 'Leak' if labels_dict['training'][idx] == 0 else 'No-leak'

    plt.title(f"Denoised: {label_name}")
    plt.grid()

    # Señal normalizada entre 0 y 1
    plt.subplot(rows, cols, i*2+2)
    plt.plot(normalized_signals_dict['training'][idx])
    plt.title(f"Normalized: {label_name}")
    plt.ylim([-0.1, 1.1])  # Ajustar límites para visualizar mejor la normalización
    plt.grid()

plt.tight_layout()
plt.show()

# Usar las señales normalizadas para los pasos siguientes
wavelet_denoised_signals_dict = normalized_signals_dict

## 4. Cálculo de Escalogramas CWT

In [ ]:
def calculate_cwt_with_coi(signal, fs=25600, f0=1.0, f1=None, fn=20,
                         sigma=6.0, fast=True, norm=True, scaling="log", nthreads=8,
                         calculate_coi=True):
    """
    Calcula la CWT de una señal completa utilizando la biblioteca fCWT con manejo opcional del COI.

    Args:
        signal: Señal completa a procesar
        fs: Frecuencia de muestreo
        f0: Frecuencia mínima
        f1: Frecuencia máxima (si es None, se usa fs/2)
        fn: Número de escalas de frecuencia (20 por defecto)
        sigma: Parámetro de la wavelet Morlet
        fast: Usar algoritmo rápido de CWT
        norm: Normalizar la salida
        scaling: Tipo de escalado ("log", "lin", "pow")
        nthreads: Número de hilos para el cálculo paralelo
        calculate_coi: Si es True, calcula la máscara COI; si es False, devuelve una máscara de todos True

    Returns:
        freqs_array: Array de frecuencias
        cwt_output: Matriz de coeficientes CWT
        coi_mask: Máscara del COI
        scales_array: Array de escalas
    """
    # Verificar tipo de datos y convertir si es necesario
    signal = np.array(signal, dtype=np.float32)

    # Establecer frecuencia máxima si no se especifica
    if f1 is None:
        f1 = fs/2

    # Inicializar la wavelet Morlet con el sigma especificado
    morlet = fcwt.Morlet(sigma)

    # Configurar escalas según el tipo de escalado
    if scaling.lower() == "log":
        scale_type = fcwt.FCWT_LOGSCALES
    else:
        scale_type = fcwt.FCWT_LINFREQS

    # Inicializar escalas como objeto
    scales_obj = fcwt.Scales(morlet, scale_type, fs, f0, f1, fn)

    # Extraer valores de escalas y frecuencias a arrays de NumPy
    # Crear arrays para almacenar los valores
    scales_array = np.zeros(fn, dtype=np.float32)
    freqs_array = np.zeros(fn, dtype=np.float32)

    # Llenar los arrays con los valores - estos métodos modifican los arrays pasados
    scales_obj.getScales(scales_array)
    scales_obj.getFrequencies(freqs_array)

    # Inicializar objeto FCWT
    fcwt_obj = fcwt.FCWT(morlet, nthreads, fast, norm)

    # Inicializar matriz de salida
    cwt_output = np.zeros((fn, len(signal)), dtype=np.complex64)

    # Calcular CWT - usar el objeto scales_obj directamente, no el array
    fcwt_obj.cwt(signal, scales_obj, cwt_output)

    # Calcular la máscara del COI solo si se solicita
    if calculate_coi:
        coi_mask = calculate_coi_mask(cwt_output, scales_array, len(signal), sigma)
    else:
        # Si no se requiere COI, crear una máscara de todos True (no se aplica enmascaramiento)
        coi_mask = np.ones_like(cwt_output, dtype=bool)

    return freqs_array, cwt_output, coi_mask, scales_array

def calculate_coi_mask(cwt_output, scales, signal_length, sigma=6.0):
    """
    Calcula la máscara del Cone of Influence (COI) para un escalograma CWT.
    Ahora 'scales' es un array de NumPy, no un objeto SwigPyObject.
    """
    mask = np.ones_like(cwt_output, dtype=bool)

    # Calcular la máscara del COI para cada escala
    for i in range(len(scales)):
        scale = scales[i]  # Acceder a cada valor individualmente

        # El ancho del borde es proporcional a la escala y sigma
        border_width = int(np.ceil(sigma * np.sqrt(2) * scale))

        # Limitar el ancho del borde
        border_width = min(border_width, signal_length // 2)

        # Marcar regiones del COI como False
        if border_width > 0:
            mask[i, :border_width] = False
            mask[i, -border_width:] = False

    return mask


def get_cwt_features_segmented(signals, labels, segment_size=512, overlap=0, fs=25600, 
                              f0=1.0, f1=None, fn=10, sigma=6.0, nthreads=8, 
                              scaling="log", fast=True, norm=True, apply_coi=True):
    """
    Calcula escalogramas CWT para segmentos de señales usando fCWT con aplicación opcional de la máscara del COI.
    
    Args:
        signals: Lista de señales completas a procesar
        labels: Lista de etiquetas correspondientes
        segment_size: Tamaño de cada segmento en muestras
        overlap: Solapamiento entre segmentos consecutivos en muestras
        fs: Frecuencia de muestreo en Hz
        f0: Frecuencia mínima
        f1: Frecuencia máxima (si es None, se usa fs/2)
        fn: Número de escalas de frecuencia
        sigma: Parámetro de la wavelet Morlet
        nthreads: Número de hilos para el cálculo paralelo
        scaling: Tipo de escalado ("log", "lin", "pow")
        fast: Usar algoritmo rápido de CWT
        norm: Normalizar la salida
        apply_coi: Si es True, aplica la máscara COI; si es False, no la aplica
        
    Returns:
        scalograms: Lista de escalogramas
        scalogram_labels: Lista de etiquetas
        signal_indices: Lista de índices que indican a qué señal original pertenece cada segmento
        segment_indices: Lista de índices que indican el orden del segmento dentro de su señal original
        coi_percentages: Lista de porcentajes de coeficientes dentro del COI
        freqs_array: Array de frecuencias utilizadas en el cálculo CWT
    """
    # Configuración inicial
    if f1 is None:
        f1 = fs/2  # Frecuencia de Nyquist

    # Estructuras para almacenar resultados
    scalograms = []
    scalogram_labels = []
    signal_indices = []  # A qué señal pertenece cada segmento
    segment_indices = []  # Orden del segmento dentro de su señal
    coi_percentages = []
    freqs_array = None  # Para almacenar las frecuencias

    print(f"Calculando escalogramas por segmentos con wavelet Morlet (sigma={sigma})")
    print(f"Rango de frecuencias: {f0} - {f1} Hz, {fn} bandas")
    print(f"Procesando segmentos de {segment_size} muestras, solapamiento: {overlap}")
    print(f"Aplicar máscara COI: {'Sí' if apply_coi else 'No'}")

    # Procesar cada señal
    for signal_idx, (signal, label) in enumerate(tqdm(zip(signals, labels), total=len(signals), desc="Procesando señales")):
        # Calcular número de segmentos
        stride = segment_size - overlap
        n_segments = max(1, (len(signal) - segment_size) // stride + 1)
        
        # Procesar cada segmento
        for segment_idx in range(n_segments):
            start = segment_idx * stride
            end = start + segment_size
            
            # Extraer segmento
            segment = signal[start:end]
            
            # Si el último segmento es más corto, rellenarlo con ceros
            if len(segment) < segment_size:
                segment = np.pad(segment, (0, segment_size - len(segment)), 'constant')
            
            try:
                # Calcular CWT con manejo del COI
                freqs, cwt_coef, coi_mask, scales = calculate_cwt_with_coi(
                    segment, fs, f0, f1, fn, sigma, fast, norm, scaling, nthreads, 
                    calculate_coi=apply_coi
                )
                
                # Guardar las frecuencias del primer cálculo (son iguales para todos los segmentos)
                if freqs_array is None:
                    freqs_array = freqs

                # Calcular porcentaje de coeficientes dentro del COI válido
                if apply_coi:
                    valid_percentage = np.mean(coi_mask) * 100
                else:
                    valid_percentage = 100.0  # 100% si no se aplica COI
                
                coi_percentages.append(valid_percentage)

                # Crear escalograma completo (intensidad de la transformada)
                scalogram_full = np.square(np.abs(cwt_coef)).T

                # Crear escalograma con COI aplicado (ceros en los bordes afectados)
                if apply_coi:
                    scalogram_coi = scalogram_full.copy()
                    scalogram_coi[~coi_mask.T] = 0
                else:
                    scalogram_coi = scalogram_full  # Sin máscara, usar directamente el escalograma completo

                # Normalizar escalogramas individualmente
                # max_val_full = np.max(scalogram_full)
                # if max_val_full > 0:
                #     scalogram_full = scalogram_full / max_val_full
                #     if apply_coi and id(scalogram_coi) != id(scalogram_full):
                #         scalogram_coi = scalogram_coi / max_val_full

                # Preparar para entrenamiento (añadir dimensión de canal)
                training_scalogram = scalogram_coi.reshape(scalogram_coi.shape + (1,))

                # Almacenar resultados
                scalograms.append(training_scalogram)
                scalogram_labels.append(label)
                signal_indices.append(signal_idx)
                segment_indices.append(segment_idx)

            except Exception as e:
                print(f"Error en señal {signal_idx}, segmento {segment_idx}: {e}")
                continue

    # Mostrar información sobre los escalogramas generados
    print(f"Escalogramas calculados: {len(scalograms)}")
    print(f"Número de señales originales: {len(signals)}")
    if len(scalograms) > 0:
        print(f"Forma de un escalograma: {scalograms[0].shape}")
    
    return scalograms, scalogram_labels, signal_indices, segment_indices, coi_percentages, freqs_array

# Función para agrupar escalogramas por señal original
def group_scalograms_by_signal(scalograms, labels, signal_indices, segment_indices):
    """
    Agrupa escalogramas por señal original y los ordena por el índice del segmento.
    
    Args:
        scalograms: Lista de escalogramas
        labels: Lista de etiquetas correspondientes
        signal_indices: Lista de índices de señal para cada escalograma
        segment_indices: Lista de índices de segmento para cada escalograma
        
    Returns:
        grouped_scalograms: Lista de listas de escalogramas agrupados por señal original
        grouped_labels: Lista de etiquetas correspondientes a cada grupo
    """
    # Determinar número de señales únicas
    unique_signal_indices = sorted(set(signal_indices))
    
    # Estructuras para almacenar resultados
    grouped_scalograms = []
    grouped_labels = []
    
    for signal_idx in unique_signal_indices:
        # Encontrar todos los escalogramas de esta señal
        indices = [i for i, si in enumerate(signal_indices) if si == signal_idx]
        
        # Extraer escalogramas, segmentos y etiquetas
        signal_scalograms = [scalograms[i] for i in indices]
        signal_segment_indices = [segment_indices[i] for i in indices]
        
        # Ordenar por índice de segmento
        ordered_indices = np.argsort(signal_segment_indices)
        ordered_scalograms = [signal_scalograms[i] for i in ordered_indices]
        
        # La etiqueta es la misma para todos los segmentos de una señal
        signal_label = labels[indices[0]]
        
        # Almacenar resultados
        grouped_scalograms.append(ordered_scalograms)
        grouped_labels.append(signal_label)
    
    return grouped_scalograms, grouped_labels


## 4.2. Visualizando los escalogramas

In [ ]:
# Función para visualizar escalogramas con frecuencias adecuadas
def plot_cwt_scalograms_with_freqs(scalograms, labels, label_codes_dict, freqs_array, num_per_class=1, 
                                  fs=25600, title="Escalogramas CWT", figsize=(15, 10)):
    """
    Visualiza escalogramas CWT con las frecuencias exactas utilizadas en el cálculo.
    
    Args:
        scalograms: Lista de escalogramas CWT
        labels: Lista de etiquetas correspondientes
        label_codes_dict: Diccionario de códigos de etiquetas
        freqs_array: Array de frecuencias usadas en el cálculo CWT
        num_per_class: Número de escalogramas a mostrar por clase
        fs: Frecuencia de muestreo (Hz)
        title: Título de la figura
        figsize: Tamaño de la figura (ancho, alto)
    """
    # Obtener etiquetas únicas
    unique_labels = np.unique(labels)
    num_classes = len(unique_labels)
    
    # Configurar subplot grid
    rows = num_classes
    cols = num_per_class
    
    # Crear figura
    plt.figure(figsize=figsize)
    
    for i, label_code in enumerate(unique_labels):
        # Obtener índices de esta clase
        class_indices = [idx for idx, l in enumerate(labels) if l == label_code]
        
        if len(class_indices) == 0:
            continue
            
        # Seleccionar ejemplos aleatorios
        selected_indices = random.sample(class_indices, min(num_per_class, len(class_indices)))
        
        for j, idx in enumerate(selected_indices):
            # Obtener etiqueta en texto
            if 'classification_mode' in globals() and globals()['classification_mode'] == 'five_classes':
                label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(label_code)]
            else:
                label_name = 'Leak' if label_code == 0 else 'No-leak'
                
            # Obtener escalograma
            scalogram = np.squeeze(scalograms[idx])
            
            # Crear subplot
            ax = plt.subplot(rows, cols, i*cols + j + 1)
            
            # Calcular valores para el eje de tiempo
            num_samples = scalogram.shape[0]
            duration = num_samples / fs  # duración en segundos
            
            # Crear puntos para las etiquetas del eje de tiempo (en segundos)
            time_ticks = np.linspace(0, duration, 5)  # 5 marcas de tiempo
            time_pos = np.linspace(0, num_samples, 5)  # posiciones correspondientes
            
            # Mostrar escalograma con orientación correcta (frecuencias en el eje Y)
            im = ax.imshow(scalogram.T, aspect='auto', origin='lower', 
                         cmap='viridis', interpolation='nearest')
            
            # Configurar eje y logarítmico
            ax.set_yscale('log')
            
            # Ajustar los límites de los ejes
            ax.set_xlim(0, num_samples)
            
            # Configurar etiquetas de tiempo
            ax.set_xticks(time_pos)
            ax.set_xticklabels([f"{t:.2f}" for t in time_ticks], fontsize=10)
            
            # Seleccionar algunas frecuencias para mostrar en el eje Y
            # Definir puntos logarítmicos para las etiquetas de frecuencia
            if min(freqs_array) < 1:
                freq_points = np.array([1, 10, 100, 1000, 5000, 10000])
            else:
                freq_points = np.array([1, 10, 100, 1000, 2000, 5000, 10000])
            
            # Filtrar frecuencias que están fuera del rango calculado
            freq_points = freq_points[(freq_points >= min(freqs_array)) & (freq_points <= max(freqs_array))]
            
            # Encontrar los índices más cercanos a estas frecuencias
            y_ticks = []
            y_labels = []
            
            for freq in freq_points:
                # Encontrar el índice más cercano en freqs_array
                idx = np.argmin(np.abs(freqs_array - freq))
                y_ticks.append(idx)
                y_labels.append(f"{freqs_array[idx]:.0f}")
            
            # Configurar etiquetas del eje Y
            ax.set_yticks(y_ticks)
            ax.set_yticklabels(y_labels, fontsize=10)
            
            # Configurar etiquetas y título
            ax.set_title(f"{label_name}", fontsize=12)
            ax.set_xlabel('Tiempo (s)', fontsize=10)
            ax.set_ylabel('Frecuencia (Hz)', fontsize=10)
            
            # Añadir barra de colores
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# Función para visualizar secuencia de escalogramas de una misma señal
def plot_scalogram_sequence(signal_scalograms, label, label_codes_dict, freqs_array, 
                          fs=25600, segment_size=512, max_segments=6, figsize=(18, 10)):
    """
    Visualiza una secuencia de escalogramas que pertenecen a la misma señal original.
    
    Args:
        signal_scalograms: Lista de escalogramas de una misma señal
        label: Etiqueta de la señal
        label_codes_dict: Diccionario de códigos de etiquetas
        freqs_array: Array de frecuencias usadas en el cálculo CWT
        fs: Frecuencia de muestreo (Hz)
        segment_size: Tamaño de cada segmento
        max_segments: Máximo número de segmentos a mostrar
        figsize: Tamaño de la figura
    """
    # Obtener etiqueta en texto
    if 'classification_mode' in globals() and globals()['classification_mode'] == 'five_classes':
        label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(label)]
    else:
        label_name = 'Leak' if label == 0 else 'No-leak'
    
    # Limitar número de segmentos a mostrar
    n_segments = min(len(signal_scalograms), max_segments)
    signal_scalograms = signal_scalograms[:n_segments]
    
    # Crear figura
    fig, axes = plt.subplots(1, n_segments, figsize=figsize)
    
    if n_segments == 1:
        axes = [axes]  # Convertir a lista si solo hay un subplot
    
    # Duración de cada segmento en segundos
    segment_duration = segment_size / fs
    
    for i, (ax, scalogram) in enumerate(zip(axes, signal_scalograms)):
        # Obtener escalograma
        scalogram = np.squeeze(scalogram)
        
        # Mostrar escalograma
        im = ax.imshow(scalogram.T, aspect='auto', origin='lower', 
                     cmap='viridis', interpolation='nearest')
        
        # Configurar eje y logarítmico
        ax.set_yscale('log')
        
        # Tiempo de inicio de este segmento
        start_time = i * segment_duration
        
        # Crear puntos para las etiquetas del eje de tiempo (en segundos)
        time_ticks = np.linspace(0, segment_duration, 3)
        time_labels = [f"{start_time + t:.2f}" for t in time_ticks]
        
        # Configurar etiquetas de tiempo
        ax.set_xticks(np.linspace(0, segment_size, 3))
        ax.set_xticklabels(time_labels, fontsize=9)
        
        # Seleccionar algunas frecuencias para mostrar en el eje Y
        # Definir puntos logarítmicos para las etiquetas de frecuencia
        if min(freqs_array) < 1:
            freq_points = np.array([1, 10, 100, 1000, 10000])
        else:
            freq_points = np.array([10, 100, 1000, 10000])
        
        # Filtrar frecuencias que están fuera del rango calculado
        freq_points = freq_points[(freq_points >= min(freqs_array)) & (freq_points <= max(freqs_array))]
        
        # Encontrar los índices más cercanos a estas frecuencias
        y_ticks = []
        y_labels = []
        
        for freq in freq_points:
            # Encontrar el índice más cercano en freqs_array
            idx = np.argmin(np.abs(freqs_array - freq))
            y_ticks.append(idx)
            y_labels.append(f"{freqs_array[idx]:.0f}")
        
        # Configurar etiquetas del eje Y
        ax.set_yticks(y_ticks)
        ax.set_yticklabels(y_labels, fontsize=9)
        
        # Título para cada segmento
        ax.set_title(f"Segmento {i+1}", fontsize=10)
        
        # Etiquetas solo para el primer y último subplot
        if i == 0:
            ax.set_ylabel('Frecuencia (Hz)', fontsize=10)
        
        if i == n_segments - 1:
            # Añadir barra de colores al último subplot
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    # Título general
    plt.suptitle(f"Secuencia de escalogramas - {label_name}", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.94])
    plt.show()

## 4.3 Creando y visualizando los escalogramas

In [ ]:
# Calcular escalogramas CWT segmentados
print("\n=== Calculando escalogramas CWT segmentados (20 escalas) ===")

# Tamaño de segmento
segment_size = 512
overlap = 0  # Sin solapamiento entre segmentos
fs=25600  # Frecuencia de muestreo
f0=1.0  # Frecuencia mínima
# Conjunto de entrenamiento
train_scalograms, train_labels, train_signal_indices, train_segment_indices, train_coi_percentages, train_freqs = get_cwt_features_segmented(
    wavelet_denoised_signals_dict['training'],
    labels_dict['training'],
    segment_size=segment_size,
    overlap=overlap,
    fn=10,
    sigma=6.0,
    scaling="log",
    nthreads=1,
    apply_coi=True  # Configurar según necesidad
)

# Conjunto de prueba
test_scalograms, test_labels, test_signal_indices, test_segment_indices, test_coi_percentages, test_freqs = get_cwt_features_segmented(
    wavelet_denoised_signals_dict['testing'],
    labels_dict['testing'],
    segment_size=segment_size,
    overlap=overlap,
    fn=10,
    sigma=6.0,
    scaling="log",
    nthreads=1,
    apply_coi=True # Configurar según necesidad
)

# Imprimir información
print(f"\nEscalogramas de entrenamiento: {len(train_scalograms)}, Forma: {train_scalograms[0].shape}")
print(f"Escalogramas de prueba: {len(test_scalograms)}, Forma: {test_scalograms[0].shape}")
print(f"Número de señales de entrenamiento: {len(set(train_signal_indices))}")
print(f"Número de señales de prueba: {len(set(test_signal_indices))}")
print(f"Segmentos por señal: {len(train_scalograms) // len(set(train_signal_indices))}")

# Visualizar algunos escalogramas individuales
plot_cwt_scalograms_with_freqs(
    train_scalograms, 
    train_labels, 
    label_codes_dict,
    train_freqs,
    num_per_class=3,
    fs=fs,
    title="Escalogramas CWT segmentados",
    figsize=(18, 12)
)

# Agrupar escalogramas por señal original
print("\n=== Agrupando escalogramas por señal original ===")
train_grouped_scalograms, train_grouped_labels = group_scalograms_by_signal(
    train_scalograms, train_labels, train_signal_indices, train_segment_indices
)

test_grouped_scalograms, test_grouped_labels = group_scalograms_by_signal(
    test_scalograms, test_labels, test_signal_indices, test_segment_indices
)

print(f"Señales de entrenamiento agrupadas: {len(train_grouped_scalograms)}")
print(f"Señales de prueba agrupadas: {len(test_grouped_labels)}")
print(f"Segmentos por señal: {len(train_grouped_scalograms[0])}")

# Visualizar secuencia de escalogramas de una señal
# Seleccionar una señal aleatoria con fuga y otra sin fuga
leak_indices = [i for i, label in enumerate(train_grouped_labels) if label == 0]
no_leak_indices = [i for i, label in enumerate(train_grouped_labels) if label == 1]

if leak_indices:
    leak_idx = random.choice(leak_indices)
    print(f"\nVisualizando secuencia de escalogramas de una señal con fuga (índice: {leak_idx})")
    plot_scalogram_sequence(
        train_grouped_scalograms[leak_idx],
        train_grouped_labels[leak_idx],
        label_codes_dict,
        train_freqs,
        fs=fs,
        segment_size=segment_size,
        max_segments=8,
        figsize=(20, 6)
    )

if no_leak_indices:
    no_leak_idx = random.choice(no_leak_indices)
    print(f"\nVisualizando secuencia de escalogramas de una señal sin fuga (índice: {no_leak_idx})")
    plot_scalogram_sequence(
        train_grouped_scalograms[no_leak_idx],
        train_grouped_labels[no_leak_idx],
        label_codes_dict,
        train_freqs,
        fs=fs,
        segment_size=segment_size,
        max_segments=8,
        figsize=(20, 6)
    )

## 5. Procesamiento de Escalogramas - Creación de ELIS

In [ ]:
# Implementación de Non-Local Means (NLM)
def apply_nlm(image, h=10, template_window_size=7, search_window_size=21):
    """
    Aplica el algoritmo Non-Local Means para reducir ruido

    Args:
        image: Imagen de entrada
        h: Parámetro de filtrado. Mayor valor = más suavizado
        template_window_size: Tamaño de la ventana de comparación
        search_window_size: Tamaño de la ventana de búsqueda

    Returns:
        Imagen con ruido reducido
    """
    # Convertir a uint8 para OpenCV
    normalized_img = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Aplicar NLM
    denoised = cv2.fastNlMeansDenoising(
        normalized_img,
        None,
        h=h,
        templateWindowSize=template_window_size,
        searchWindowSize=search_window_size
    )

    # Normalizar de nuevo entre 0 y 1
    return denoised / 255.0

# Implementación de Adaptive Histogram Equalization (AHE)
def apply_clahe(image, clip_limit=2.0, grid_size=(8, 8)):
    """
    Aplica Contrast Limited Adaptive Histogram Equalization (CLAHE)

    Args:
        image: Imagen de entrada
        clip_limit: Límite de contraste
        grid_size: Tamaño de la cuadrícula

    Returns:
        Imagen con contraste mejorado
    """
    # Convertir a uint8 para OpenCV
    normalized_img = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

    # Crear objeto CLAHE
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)

    # Aplicar CLAHE
    equalized = clahe.apply(normalized_img)

    # Normalizar de nuevo entre 0 y 1
    return equalized / 255.0

def generate_elis(scalograms, batch_size=50):
    """
    Genera ELIS (Enhanced Leak-Induced Scalograms) aplicando NLM y AHE

    Args:
        scalograms: Array de escalogramas de forma [batch, height, width, channels]
        batch_size: Tamaño del lote para procesar
        
    Returns:
        ELIS: Escalogramas mejorados
    """
    elis_scalograms = []
    
    # Procesar por lotes para evitar problemas de memoria
    num_batches = (len(scalograms) + batch_size - 1) // batch_size
    
    for batch_idx in tqdm(range(num_batches), desc="Generando ELIS"):
        # Obtener el lote actual
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, len(scalograms))
        batch = scalograms[start_idx:end_idx]
        
        batch_elis = []
        for scalogram in batch:
            # Quitar dimensión de canal si existe
            if len(scalogram.shape) == 3:
                scalogram = np.squeeze(scalogram)
            
            # 1. Aplicar Non-Local Means para reducir ruido
            nlm_scalogram = apply_nlm(scalogram, h=10)
            
            # 2. Aplicar CLAHE para mejorar contraste
            elis = apply_clahe(nlm_scalogram, clip_limit=2.0)
            
            # Añadir dimensión de canal
            elis = np.expand_dims(elis, axis=-1)
            
            batch_elis.append(elis)
        
        # Concatenar el lote procesado
        elis_scalograms.extend(batch_elis)
    
    return np.array(elis_scalograms)

# Aplicar ELIS a grupos de escalogramas
def apply_elis_to_grouped(grouped_scalograms, batch_size=50):
    """
    Aplica el proceso ELIS a grupos de escalogramas manteniendo la estructura de grupos

    Args:
        grouped_scalograms: Lista de listas de escalogramas
        batch_size: Tamaño del lote para procesamiento
        
    Returns:
        elis_grouped: Lista de listas de escalogramas ELIS
    """
    elis_grouped = []
    
    for i, group in enumerate(tqdm(grouped_scalograms, desc="Procesando grupos")):
        # Procesar cada escalograma en el grupo
        group_elis = []
        for scalogram in group:
            # Quitar dimensión de canal si existe
            if len(scalogram.shape) == 3:
                scalogram_2d = np.squeeze(scalogram)
            else:
                scalogram_2d = scalogram
            
            # 1. Aplicar Non-Local Means para reducir ruido
            nlm_scalogram = apply_nlm(scalogram_2d, h=10)
            
            # 2. Aplicar CLAHE para mejorar contraste
            elis = apply_clahe(nlm_scalogram, clip_limit=2.0)
            
            # Añadir dimensión de canal
            elis = np.expand_dims(elis, axis=-1)
            
            group_elis.append(elis)
        
        elis_grouped.append(group_elis)
    
    return elis_grouped

# Visualización de escalogramas ELIS
def visualize_elis_comparison(original_scalograms, elis_scalograms, labels, label_codes_dict, 
                             num_examples=2, freqs_array=None, fs=25600):
    """
    Visualiza una comparación entre escalogramas originales y ELIS
    
    Args:
        original_scalograms: Lista de escalogramas originales
        elis_scalograms: Lista de escalogramas ELIS
        labels: Lista de etiquetas
        label_codes_dict: Diccionario de códigos de etiquetas
        num_examples: Número de ejemplos a mostrar por clase
        freqs_array: Array de frecuencias (opcional)
        fs: Frecuencia de muestreo
    """
    unique_labels = np.unique(labels)
    fig, axes = plt.subplots(len(unique_labels), 2*num_examples, figsize=(5*num_examples, 4*len(unique_labels)))
    
    if len(unique_labels) == 1:
        axes = np.expand_dims(axes, axis=0)
    
    for i, label_code in enumerate(unique_labels):
        # Encontrar índices de esta clase
        class_indices = [j for j, l in enumerate(labels) if l == label_code]
        if len(class_indices) == 0:
            continue
        
        # Seleccionar ejemplos aleatorios
        selected_indices = random.sample(class_indices, min(num_examples, len(class_indices)))
        
        for j, idx in enumerate(selected_indices):
            # Obtener etiqueta de texto
            if 'classification_mode' in globals() and globals()['classification_mode'] == 'five_classes':
                label_name = list(label_codes_dict.keys())[list(label_codes_dict.values()).index(label_code)]
            else:
                label_name = 'Leak' if label_code == 0 else 'No-leak'
            
            # Escalograma original
            ax = axes[i, j*2]
            
            # Quitar dimensión de canal si existe
            orig_img = np.squeeze(original_scalograms[idx])
            
            im = ax.imshow(orig_img, aspect='auto', origin='lower', cmap='viridis')
            
            # Configurar eje y logarítmico si tenemos frecuencias
            if freqs_array is not None:
                ax.set_yscale('log')
                
                # Seleccionar algunas frecuencias para mostrar
                if min(freqs_array) < 1:
                    freq_points = np.array([1, 10, 100, 1000, 5000, 10000])
                else:
                    freq_points = np.array([1, 10, 100, 1000, 2000, 5000, 10000])
                
                # Filtrar frecuencias fuera del rango
                freq_points = freq_points[(freq_points >= min(freqs_array)) & (freq_points <= max(freqs_array))]
                
                # Encontrar índices más cercanos
                y_ticks = []
                y_labels = []
                
                for freq in freq_points:
                    idx = np.argmin(np.abs(freqs_array - freq))
                    y_ticks.append(idx)
                    y_labels.append(f"{freqs_array[idx]:.0f}")
                
                # Configurar etiquetas de frecuencia
                ax.set_yticks(y_ticks)
                ax.set_yticklabels(y_labels, fontsize=9)
            
            ax.set_title(f"Original: {label_name}")
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            
            # Escalograma ELIS
            ax = axes[i, j*2+1]
            
            # Quitar dimensión de canal si existe
            elis_img = np.squeeze(elis_scalograms[idx])
            
            im = ax.imshow(elis_img, aspect='auto', origin='lower', cmap='viridis')
            
            # Configurar eje y logarítmico si tenemos frecuencias
            if freqs_array is not None:
                ax.set_yscale('log')
                ax.set_yticks(y_ticks)
                ax.set_yticklabels(y_labels, fontsize=9)
            
            ax.set_title(f"ELIS: {label_name}")
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    
    plt.tight_layout()
    plt.show()

## 5.1 Aplicando ELIS a los escalogramas

In [ ]:
# Aplicar ELIS a los escalogramas agrupados
print("\n=== Aplicando ELIS a los escalogramas agrupados ===")

# Aplicar al conjunto de entrenamiento 
print("Procesando conjunto de entrenamiento...")
train_elis_grouped = apply_elis_to_grouped(train_grouped_scalograms)

# Aplicar al conjunto de prueba
print("Procesando conjunto de prueba...")
test_elis_grouped = apply_elis_to_grouped(test_grouped_scalograms)

print(f"Grupos ELIS de entrenamiento: {len(train_elis_grouped)}")
print(f"Grupos ELIS de prueba: {len(test_elis_grouped)}")

# Visualizar comparación de escalogramas originales vs ELIS
# Seleccionar un grupo de cada clase para visualizar
leak_indices = [i for i, label in enumerate(train_grouped_labels) if label == 0]
no_leak_indices = [i for i, label in enumerate(train_grouped_labels) if label == 1]

if leak_indices:
    leak_idx = random.choice(leak_indices)
    print(f"\nComparando escalogramas originales vs ELIS para señal con fuga (índice: {leak_idx})")
    # Seleccionar el primer segmento del grupo
    visualize_elis_comparison(
        [train_grouped_scalograms[leak_idx][0], train_grouped_scalograms[leak_idx][1]], 
        [train_elis_grouped[leak_idx][0], train_elis_grouped[leak_idx][1]],
        [train_grouped_labels[leak_idx], train_grouped_labels[leak_idx]],
        label_codes_dict,
        num_examples=1,
        freqs_array=train_freqs
    )

if no_leak_indices:
    no_leak_idx = random.choice(no_leak_indices)
    print(f"\nComparando escalogramas originales vs ELIS para señal sin fuga (índice: {no_leak_idx})")
    # Seleccionar el primer segmento del grupo
    visualize_elis_comparison(
        [train_grouped_scalograms[no_leak_idx][0], train_grouped_scalograms[no_leak_idx][1]], 
        [train_elis_grouped[no_leak_idx][0], train_elis_grouped[no_leak_idx][1]],
        [train_grouped_labels[no_leak_idx], train_grouped_labels[no_leak_idx]],
        label_codes_dict,
        num_examples=1,
        freqs_array=train_freqs
    )

## 6. Preparación de Datos para el Modelo Secuencial

In [ ]:
def prepare_data_for_sequence_model(grouped_scalograms, grouped_labels, classification_mode, max_segments=None):
    """
    Prepara los datos de escalogramas agrupados para el modelo secuencial.
    
    Args:
        grouped_scalograms: Lista de listas de escalogramas agrupados por señal original
        grouped_labels: Lista de etiquetas correspondientes a cada grupo
        classification_mode: Modo de clasificación ('five_classes' o 'binary')
        max_segments: Número máximo de segmentos a incluir por señal. Si es None, usa todos.
        
    Returns:
        x_data: Array numpy de escalogramas con forma [n_señales, n_segmentos, altura, ancho, canales]
        y_data: Array numpy de etiquetas originales
        y_onehot: Array numpy de etiquetas en formato one-hot
        num_classes: Número de clases
    """
    # Determinar el número máximo de segmentos por señal si no se especifica
    if max_segments is None:
        max_segments = max([len(group) for group in grouped_scalograms])
    
    # Determinar la forma de un escalograma individual
    sample_scalogram = grouped_scalograms[0][0]
    scalogram_shape = sample_scalogram.shape
    
    # Crear arrays para almacenar datos y padding según sea necesario
    n_signals = len(grouped_scalograms)
    height, width = scalogram_shape[0], scalogram_shape[1]
    channels = 1 if len(scalogram_shape) == 3 else scalogram_shape[-1]
    
    # Inicializar array para los datos X
    x_data = np.zeros((n_signals, max_segments, height, width, channels))
    
    # Llenar el array con los escalogramas, aplicando padding si es necesario
    for i, group in enumerate(grouped_scalograms):
        # Limitar al número máximo de segmentos
        n_segs = min(len(group), max_segments)
        
        # Copiar escalogramas al array
        for j in range(n_segs):
            scalogram = group[j]
            # Asegurarse de que el escalograma tenga la dimensión de canal
            if len(scalogram.shape) == 2:
                scalogram = np.expand_dims(scalogram, axis=-1)
            x_data[i, j] = scalogram
    
    # Convertir etiquetas a arrays numpy
    y_data = np.array(grouped_labels)
    
    # Determinar número de clases según el modo
    if classification_mode == 'five_classes':
        num_classes = 5
    else:  # binary
        num_classes = 2
    
    # One-hot encoding de las etiquetas
    y_onehot = tf.keras.utils.to_categorical(y_data, num_classes)
    
    return x_data, y_data, y_onehot, num_classes

def train_val_test_split_by_signal(grouped_scalograms, grouped_labels, test_size=0.2, val_size=0.2, random_state=42):
    """
    Divide los datos agrupados en conjuntos de entrenamiento, validación y prueba,
    manteniendo juntos los escalogramas de una misma señal.
    
    Args:
        grouped_scalograms: Lista de listas de escalogramas agrupados por señal original
        grouped_labels: Lista de etiquetas correspondientes a cada grupo
        test_size: Proporción de datos para prueba
        val_size: Proporción de datos para validación (del conjunto de entrenamiento)
        random_state: Semilla para reproducibilidad
        
    Returns:
        train_grouped_scalograms, val_grouped_scalograms, test_grouped_scalograms,
        train_grouped_labels, val_grouped_labels, test_grouped_labels
    """
    # Convertir a arrays numpy para facilitar la división
    grouped_scalograms_np = np.array(grouped_scalograms, dtype=object)
    grouped_labels_np = np.array(grouped_labels)
    
    # Primera división: separar conjunto de prueba
    train_val_scalograms, test_grouped_scalograms, train_val_labels, test_grouped_labels = train_test_split(
        grouped_scalograms_np,
        grouped_labels_np,
        test_size=test_size,
        stratify=grouped_labels_np,  # Para mantener la proporción de clases
        random_state=random_state
    )
    
    # Segunda división: separar conjuntos de entrenamiento y validación
    train_grouped_scalograms, val_grouped_scalograms, train_grouped_labels, val_grouped_labels = train_test_split(
        train_val_scalograms,
        train_val_labels,
        test_size=val_size,
        stratify=train_val_labels,  # Para mantener la proporción de clases
        random_state=random_state
    )
    
    # Convertir de vuelta a listas Python
    train_grouped_scalograms = list(train_grouped_scalograms)
    val_grouped_scalograms = list(val_grouped_scalograms)
    test_grouped_scalograms = list(test_grouped_scalograms)
    train_grouped_labels = list(train_grouped_labels)
    val_grouped_labels = list(val_grouped_labels)
    test_grouped_labels = list(test_grouped_labels)
    
    return (train_grouped_scalograms, val_grouped_scalograms, test_grouped_scalograms,
            train_grouped_labels, val_grouped_labels, test_grouped_labels)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
print("\n=== Dividiendo datos agrupados en conjuntos de entrenamiento, validación y prueba ===")

(train_grouped_scalograms_final, val_grouped_scalograms, test_grouped_scalograms_final,
 train_grouped_labels_final, val_grouped_labels, test_grouped_labels_final) = train_val_test_split_by_signal(
    train_grouped_elis,
    train_grouped_labels,
    test_size=0.2,  # 20% para prueba
    val_size=0.2,   # 20% del resto para validación
    random_state=42
)

print(f"Señales para entrenamiento: {len(train_grouped_scalograms_final)}")
print(f"Señales para validación: {len(val_grouped_scalograms)}")
print(f"Señales para prueba: {len(test_grouped_scalograms_final)}")

# Preparar datos para el modelo secuencial
# Determinar el número máximo de segmentos por señal
max_segments = max([len(group) for group in train_grouped_scalograms_final + val_grouped_scalograms + test_grouped_scalograms_final])
print(f"Número máximo de segmentos por señal: {max_segments}")

# Preparar datos con forma [n_señales, n_segmentos, altura, ancho, canales]
x_train, y_train, y_train_onehot, num_classes = prepare_data_for_sequence_model(
    train_grouped_scalograms_final,
    train_grouped_labels_final,
    classification_mode,
    max_segments
)

x_val, y_val, y_val_onehot, _ = prepare_data_for_sequence_model(
    val_grouped_scalograms,
    val_grouped_labels,
    classification_mode,
    max_segments
)

x_test, y_test, y_test_onehot, _ = prepare_data_for_sequence_model(
    test_grouped_scalograms_final,
    test_grouped_labels_final,
    classification_mode,
    max_segments
)

print(f"\nForma de datos de entrenamiento: {x_train.shape}")
print(f"Forma de datos de validación: {x_val.shape}")
print(f"Forma de datos de prueba: {x_test.shape}")
print(f"Número de clases: {num_classes}")

## 7. Guardar dataset de ELIS en Google Drive

In [ ]:
def save_elis_dataset(train_scalograms, train_labels, test_scalograms, test_labels,
                     classification_mode, label_codes_dict,
                     output_file='/content/drive/MyDrive/Tesis/leak_detection_elis_dataset.h5'):
    """
    Guarda el dataset de ELIS en formato H5 para uso posterior

    Args:
        train_scalograms: Lista de escalogramas de entrenamiento
        train_labels: Lista de etiquetas de entrenamiento
        test_scalograms: Lista de escalogramas de prueba
        test_labels: Lista de etiquetas de prueba
        classification_mode: Modo de clasificación ('five_classes' o 'binary')
        label_codes_dict: Diccionario de códigos de etiquetas
        output_file: Ruta donde guardar el archivo H5

    Returns:
        Ruta del archivo guardado
    """
    print(f"Guardando dataset ELIS en: {output_file}")
    start_time = time.time()

    # Determinar número de clases
    num_classes = 5 if classification_mode == 'five_classes' else 2

    # Convertir a arrays numpy
    X_train = np.array(train_scalograms)
    y_train = np.array(train_labels)
    X_test = np.array(test_scalograms)
    y_test = np.array(test_labels)

    # Verificar dimensiones
    print(f"Dimensiones de X_train: {X_train.shape}")
    print(f"Dimensiones de X_test: {X_test.shape}")

    # Dividir conjunto de entrenamiento para crear un conjunto de validación
    val_split = 0.2
    val_indices = np.random.choice(len(X_train), int(len(X_train) * val_split), replace=False)
    train_mask = np.ones(len(X_train), dtype=bool)
    train_mask[val_indices] = False

    X_val = X_train[~train_mask]
    y_val = y_train[~train_mask]
    X_train_final = X_train[train_mask]
    y_train_final = y_train[train_mask]

    # Convertir etiquetas a one-hot
    y_train_onehot = tf.keras.utils.to_categorical(y_train_final, num_classes)
    y_val_onehot = tf.keras.utils.to_categorical(y_val, num_classes)
    y_test_onehot = tf.keras.utils.to_categorical(y_test, num_classes)

    # Guardar en formato H5
    with h5py.File(output_file, 'w') as hf:
        # Crear grupos
        train_group = hf.create_group('train')
        val_group = hf.create_group('val')
        test_group = hf.create_group('test')
        metadata_group = hf.create_group('metadata')

        # Guardar conjuntos de datos
        train_group.create_dataset('X_train', data=X_train_final)
        train_group.create_dataset('y_train', data=y_train_final)
        train_group.create_dataset('y_train_onehot', data=y_train_onehot)

        val_group.create_dataset('X_val', data=X_val)
        val_group.create_dataset('y_val', data=y_val)
        val_group.create_dataset('y_val_onehot', data=y_val_onehot)

        test_group.create_dataset('X_test', data=X_test)
        test_group.create_dataset('y_test', data=y_test)
        test_group.create_dataset('y_test_onehot', data=y_test_onehot)

        # Guardar metadatos
        metadata_group.attrs['num_classes'] = num_classes
        metadata_group.attrs['classification_mode'] = classification_mode
        metadata_group.attrs['label_codes_dict'] = json.dumps(label_codes_dict)

        # Guardar información de procesamiento ELIS
        processing_info = {
            'nlm_h': 10,
            'nlm_template_window': 7,
            'nlm_search_window': 21,
            'clahe_clip_limit': 2.0,
            'clahe_grid_size': [8, 8]
        }
        metadata_group.attrs['processing_info'] = json.dumps(processing_info)

    elapsed = time.time() - start_time
    print(f"Dataset ELIS guardado en {output_file}")
    print(f"Tiempo de guardado: {elapsed:.2f} segundos")
    print(f"Tamaños de los conjuntos:")
    print(f"- Entrenamiento: {X_train_final.shape}")
    print(f"- Validación: {X_val.shape}")
    print(f"- Prueba: {X_test.shape}")

    return output_file

# Guardar dataset ELIS en Google Drive
import json
elis_dataset_path = save_elis_dataset(
    train_elis_grouped,
    train_labels,
    test_elis_grouped,
    test_labels,
    classification_mode,
    label_codes_dict,
    output_file='/content/drive/MyDrive/Tesis/leak_detection_elis_dataset.h5'
)

## 8. Cargar dataset ELIS desde Google Drive

In [ ]:
def load_elis_dataset(file_path='/content/drive/MyDrive/Tesis/leak_detection_elis_dataset.h5'):
    """
    Carga el dataset ELIS desde un archivo H5

    Args:
        file_path: Ruta al archivo H5

    Returns:
        Datos de entrenamiento, validación y prueba, más metadatos
    """
    print(f"Cargando dataset ELIS desde: {file_path}")

    with h5py.File(file_path, 'r') as hf:
        # Cargar datos de entrenamiento
        X_train = np.array(hf['train']['X_train'])
        y_train = np.array(hf['train']['y_train'])
        y_train_onehot = np.array(hf['train']['y_train_onehot'])

        # Cargar datos de validación
        X_val = np.array(hf['val']['X_val'])
        y_val = np.array(hf['val']['y_val'])
        y_val_onehot = np.array(hf['val']['y_val_onehot'])

        # Cargar datos de prueba
        X_test = np.array(hf['test']['X_test'])
        y_test = np.array(hf['test']['y_test'])
        y_test_onehot = np.array(hf['test']['y_test_onehot'])

        # Cargar metadatos
        num_classes = hf['metadata'].attrs['num_classes']
        classification_mode = hf['metadata'].attrs['classification_mode']

        # Cargar diccionario de etiquetas
        if 'label_codes_dict' in hf['metadata'].attrs:
            label_codes_dict = json.loads(hf['metadata'].attrs['label_codes_dict'])
        else:
            if classification_mode == 'five_classes':
                label_codes_dict = {'Circumferential Crack': 0, 'Gasket Leak': 1,
                                  'Longitudinal Crack': 2, 'No-leak': 3, 'Orifice Leak': 4}
            else:
                label_codes_dict = {'Leak': 0, 'No-leak': 1}

        # Cargar información de procesamiento
        if 'processing_info' in hf['metadata'].attrs:
            processing_info = json.loads(hf['metadata'].attrs['processing_info'])
        else:
            processing_info = {}

    print("Dataset ELIS cargado correctamente")
    print(f"Dimensiones de datos:")
    print(f"- Entrenamiento: {X_train.shape}")
    print(f"- Validación: {X_val.shape}")
    print(f"- Prueba: {X_test.shape}")
    print(f"Modo de clasificación: {classification_mode}")
    print(f"Número de clases: {num_classes}")

    return (X_train, y_train, y_train_onehot,
            X_val, y_val, y_val_onehot,
            X_test, y_test, y_test_onehot,
            label_codes_dict, num_classes, classification_mode, processing_info)

# Para cargar el dataset (descomentar cuando se necesite)
# Descomenta las siguientes líneas si necesitas cargar un dataset previamente guardado
"""
(X_train, y_train, y_train_onehot,
 X_val, y_val, y_val_onehot,
 X_test, y_test, y_test_onehot,
 label_codes_dict, num_classes, classification_mode, processing_info) = load_elis_dataset()
"""

## 9. Implementación del Modelo ResNet-18 con Modificaciones para Escalogramas

In [ ]:
def create_resnet_block(x, filters, kernel_size=3, stride=1, use_bias=True, shortcut=False):
    """
    Crea un bloque residual para ResNet
    
    Args:
        x: Capa de entrada
        filters: Número de filtros
        kernel_size: Tamaño del kernel
        stride: Stride para la convolución
        use_bias: Si se usa bias en las capas convolucionales
        shortcut: Si es True, se añade un atajo con convolución para dimensionalidad
        
    Returns:
        Bloque residual
    """
    shortcut_x = x
    
    # Primera convolución
    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same', use_bias=use_bias,
                      kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Segunda convolución
    x = layers.Conv2D(filters, kernel_size, padding='same', use_bias=use_bias,
                      kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.BatchNormalization()(x)
    
    # Shortcut connection
    if shortcut or stride > 1:
        shortcut_x = layers.Conv2D(filters, 1, strides=stride, padding='same', use_bias=use_bias,
                                 kernel_regularizer=regularizers.l2(1e-4))(shortcut_x)
        shortcut_x = layers.BatchNormalization()(shortcut_x)
    
    # Sumar la conexión residual
    x = layers.add([x, shortcut_x])
    x = layers.ReLU()(x)
    
    return x

def create_resnet18_model(input_shape, num_classes, dropout_rate=0.5):
    """
    Crea un modelo ResNet-18 para escalogramas
    
    Args:
        input_shape: Forma de entrada (altura, anchura, canales)
        num_classes: Número de clases
        dropout_rate: Tasa de dropout
        
    Returns:
        Modelo keras
    """
    inputs = layers.Input(shape=input_shape)
    
    # Capa inicial
    x = layers.Conv2D(64, 7, strides=2, padding='same', use_bias=True,
                     kernel_regularizer=regularizers.l2(1e-4))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Bloques residuales
    x = create_resnet_block(x, 64)
    x = create_resnet_block(x, 64)
    
    x = create_resnet_block(x, 128, stride=2, shortcut=True)
    x = create_resnet_block(x, 128)
    
    x = create_resnet_block(x, 256, stride=2, shortcut=True)
    x = create_resnet_block(x, 256)
    
    x = create_resnet_block(x, 512, stride=2, shortcut=True)
    x = create_resnet_block(x, 512)
    
    # Pooling global
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dropout para regularización
    x = layers.Dropout(dropout_rate)(x)
    
    # Capa de salida
    if num_classes == 2:
        outputs = layers.Dense(1, activation='sigmoid',
                            kernel_regularizer=regularizers.l2(1e-4))(x)
    else:
        outputs = layers.Dense(num_classes, activation='softmax',
                            kernel_regularizer=regularizers.l2(1e-4))(x)
    
    return models.Model(inputs, outputs)

def create_sequence_resnet18_model(input_shape, num_classes, max_segments, dropout_rate=0.5):
    """
    Crea un modelo secuencial con ResNet-18 para procesar secuencias de escalogramas
    
    Args:
        input_shape: Forma de un escalograma individual (altura, anchura, canales)
        num_classes: Número de clases
        max_segments: Número máximo de segmentos por señal
        dropout_rate: Tasa de dropout
        
    Returns:
        Modelo keras
    """
    # Entrada para la secuencia completa [batch, n_segments, height, width, channels]
    inputs = layers.Input(shape=(max_segments,) + input_shape)
    
    # Crear modelo ResNet-18 base para un solo escalograma
    base_model = create_resnet18_model(input_shape, num_classes=num_classes, dropout_rate=dropout_rate)
    
    # Quitar la capa de salida del modelo base
    base_model_output = base_model.layers[-2].output
    feature_extractor = models.Model(inputs=base_model.inputs, outputs=base_model_output)
    
    # TimeDistributed para aplicar el mismo modelo a cada segmento
    encoded_sequence = layers.TimeDistributed(feature_extractor)(inputs)
    
    # Procesar la secuencia con LSTM bidireccional
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(encoded_sequence)
    x = layers.Bidirectional(layers.LSTM(128, dropout=0.3, recurrent_dropout=0.3))(x)
    
    # Capas densas finales con regularización
    x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
    x = layers.Dropout(dropout_rate)(x)
    
    # Capa de salida
    if num_classes == 2:
        outputs = layers.Dense(1, activation='sigmoid',
                             kernel_regularizer=regularizers.l2(1e-4))(x)
    else:
        outputs = layers.Dense(num_classes, activation='softmax',
                             kernel_regularizer=regularizers.l2(1e-4))(x)
    
    model = models.Model(inputs=inputs, outputs=outputs)
    
    # Compilar modelo
    if num_classes == 2:
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    else:
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
    
    return model

## 10. Entrenamiento del Modelo Secuencial

In [ ]:
def train_sequence_model(model, x_train, y_train, x_val, y_val, 
                         epochs=50, batch_size=16, 
                         checkpoint_dir="checkpoints", 
                         num_classes=2):
    """
    Entrena el modelo secuencial con los datos proporcionados
    
    Args:
        model: Modelo a entrenar
        x_train, y_train: Datos de entrenamiento y etiquetas
        x_val, y_val: Datos de validación y etiquetas
        epochs: Número de épocas para entrenamiento
        batch_size: Tamaño de lote
        checkpoint_dir: Directorio para guardar checkpoints
        num_classes: Número de clases
        
    Returns:
        Modelo entrenado e historial de entrenamiento
    """
    # Preparar etiquetas según el número de clases
    if num_classes == 2:
        y_train_final = y_train  # Etiquetas originales para binary_crossentropy
        y_val_final = y_val
    else:
        y_train_final = tf.keras.utils.to_categorical(y_train, num_classes)  # One-hot para categorical_crossentropy
        y_val_final = tf.keras.utils.to_categorical(y_val, num_classes)
    
    # Configurar callbacks
    os.makedirs(checkpoint_dir, exist_ok=True)
    # Definir warmup_schedule para la tasa de aprendizaje
    def warmup_schedule(epoch, lr):
        warmup_epochs = 5
        init_lr = 1e-6
        target_lr = 0.0005

        if epoch < warmup_epochs:
            return init_lr + (target_lr - init_lr) * epoch / warmup_epochs
        return lr
    
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, "best_model.h5"),
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            verbose=1,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=8,
            min_lr=1e-6,
            verbose=1
        ),
        keras.callbacks.LearningRateScheduler(warmup_schedule)
    ]
    
    # Entrenar modelo
    history = model.fit(
        x_train, y_train_final,
        validation_data=(x_val, y_val_final),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )
    
    return model, history

# Obtener la forma de un escalograma individual
individual_shape = x_train.shape[2:]  # (altura, anchura, canales)
max_segments = x_train.shape[1]  # Número máximo de segmentos

print(f"\n=== Creando y entrenando modelo secuencial ResNet-18 ===")
print(f"Forma de escalograma individual: {individual_shape}")
print(f"Número máximo de segmentos por señal: {max_segments}")
print(f"Número de clases: {num_classes}")

# Crear modelo secuencial
model = create_sequence_resnet18_model(
    input_shape=individual_shape,
    num_classes=num_classes,
    max_segments=max_segments,
    dropout_rate=0.5
)

# Mostrar resumen del modelo
model.summary()

# Entrenar modelo
checkpoint_dir = "checkpoints/resnet18_sequence_" + datetime.now().strftime("%Y%m%d_%H%M%S")
model, history = train_sequence_model(
    model=model,
    x_train=x_train,
    y_train=y_train,
    x_val=x_val,
    y_val=y_val,
    epochs=200,
    batch_size=8,  # Reducir si hay problemas de memoria
    checkpoint_dir=checkpoint_dir,
    num_classes=num_classes
)

# Guardar el modelo entrenado
model_save_path = os.path.join(checkpoint_dir, "final_model.h5")
model.save(model_save_path)
print(f"\nModelo guardado en: {model_save_path}")

## 11. Evaluación del Modelo Secuencial 

In [ ]:
def evaluate_sequence_model(model, x_test, y_test, num_classes=2):
    """
    Evalúa el modelo secuencial con el conjunto de prueba
    
    Args:
        model: Modelo entrenado
        x_test, y_test: Datos y etiquetas de prueba
        num_classes: Número de clases
        
    Returns:
        Diccionario con métricas de evaluación
    """
    # Preparar etiquetas según el número de clases
    if num_classes == 2:
        y_test_final = y_test  # Etiquetas originales para binary_crossentropy
    else:
        y_test_final = tf.keras.utils.to_categorical(y_test, num_classes)  # One-hot para categorical_crossentropy
    
    # Evaluar modelo
    print("\nEvaluando modelo en conjunto de prueba...")
    test_loss, test_acc = model.evaluate(x_test, y_test_final, verbose=1)
    print(f"Precisión en prueba: {test_acc:.4f}")
    
    # Generar predicciones
    y_pred_prob = model.predict(x_test)
    
    if num_classes == 2:
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_pred_prob, axis=1)
    
    # Calcular matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    
    # Mostrar matriz de confusión
    plt.figure(figsize=(10, 8))
    if num_classes == 2:
        class_names = ['Leak', 'No-leak']
    else:
        class_names = list(label_codes_dict.keys())
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Real')
    plt.title('Matriz de Confusión')
    plt.tight_layout()
    plt.show()
    
    # Mostrar informe de clasificación
    print("\nInforme de clasificación:")
    print(classification_report(y_test, y_pred, target_names=class_names))
    
    # Graficar precisión y pérdida durante el entrenamiento
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Validación')
    plt.title('Precisión durante entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Precisión')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.title('Pérdida durante entrenamiento')
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return {
        'test_loss': test_loss,
        'test_acc': test_acc,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_test': y_test
    }

# Evaluar modelo con conjunto de prueba
evaluation = evaluate_sequence_model(model, x_test, y_test, num_classes)

# Guardar resultados de evaluación
eval_save_path = os.path.join(checkpoint_dir, "evaluation_results.npz")
np.savez(
    eval_save_path,
    test_acc=evaluation['test_acc'],
    confusion_matrix=evaluation['confusion_matrix'],
    y_pred=evaluation['y_pred'],
    y_test=evaluation['y_test']
)
print(f"Resultados de evaluación guardados en: {eval_save_path}")

## 12.Visualización de Predicciones

In [ ]:
def visualize_predictions(model, x_test, y_test, class_names, num_examples=5):
    """
    Visualiza ejemplos de predicciones correctas e incorrectas
    
    Args:
        model: Modelo entrenado
        x_test: Datos de prueba
        y_test: Etiquetas de prueba
        class_names: Nombres de las clases
        num_examples: Número de ejemplos a visualizar
    """
    # Generar predicciones
    y_pred_prob = model.predict(x_test)
    
    if len(y_pred_prob.shape) == 2 and y_pred_prob.shape[1] > 1:  # multi-clase
        y_pred = np.argmax(y_pred_prob, axis=1)
    else:  # binario
        y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    
    # Encontrar ejemplos correctos e incorrectos
    correct_indices = np.where(y_pred == y_test)[0]
    incorrect_indices = np.where(y_pred != y_test)[0]
    
    # Visualizar ejemplos correctos
    if len(correct_indices) > 0:
        # Seleccionar algunos ejemplos aleatorios
        selected_correct = np.random.choice(correct_indices, min(num_examples, len(correct_indices)), replace=False)
        
        for idx in selected_correct:
            true_label = y_test[idx]
            pred_label = y_pred[idx]
            
            print(f"\nEjemplo correcto #{idx}")
            print(f"Etiqueta verdadera: {class_names[true_label]}")
            print(f"Predicción: {class_names[pred_label]}")
            
            # Visualizar el primer segmento no nulo de la secuencia
            sequence = x_test[idx]
            for seg_idx in range(sequence.shape[0]):
                # Verificar si el segmento tiene datos no nulos
                if np.sum(sequence[seg_idx]) > 0:
                    plt.figure(figsize=(10, 6))
                    plt.imshow(sequence[seg_idx, :, :, 0], cmap='viridis', aspect='auto')
                    plt.colorbar(label='Intensidad')
                    plt.title(f"Correcto - Verdadero: {class_names[true_label]}, Predicho: {class_names[pred_label]}")
                    plt.xlabel('Tiempo')
                    plt.ylabel('Frecuencia')
                    plt.show()
                    break
    
    # Visualizar ejemplos incorrectos
    if len(incorrect_indices) > 0:
        # Seleccionar algunos ejemplos aleatorios
        selected_incorrect = np.random.choice(incorrect_indices, min(num_examples, len(incorrect_indices)), replace=False)
        
        for idx in selected_incorrect:
            true_label = y_test[idx]
            pred_label = y_pred[idx]
            
            print(f"\nEjemplo incorrecto #{idx}")
            print(f"Etiqueta verdadera: {class_names[true_label]}")
            print(f"Predicción: {class_names[pred_label]}")
            
            # Visualizar el primer segmento no nulo de la secuencia
            sequence = x_test[idx]
            for seg_idx in range(sequence.shape[0]):
                # Verificar si el segmento tiene datos no nulos
                if np.sum(sequence[seg_idx]) > 0:
                    plt.figure(figsize=(10, 6))
                    plt.imshow(sequence[seg_idx, :, :, 0], cmap='viridis', aspect='auto')
                    plt.colorbar(label='Intensidad')
                    plt.title(f"Incorrecto - Verdadero: {class_names[true_label]}, Predicho: {class_names[pred_label]}")
                    plt.xlabel('Tiempo')
                    plt.ylabel('Frecuencia')
                    plt.show()
                    break

# Visualizar ejemplos de predicciones
if num_classes == 2:
    class_names = ['Leak', 'No-leak']
else:
    class_names = list(label_codes_dict.keys())

print("\n=== Visualizando ejemplos de predicciones ===")
visualize_predictions(model, x_test, y_test, class_names, num_examples=3)